In [2]:
# coding: utf-8
"""
corenlpを使って形態素解析を行って学習用のファイルを作成する
"""
from stanford_corenlp_pywrapper import CoreNLP
from library.filer import Filer
import re
import collections
import itertools

1. 形態素解析する600ユーザー分のレビューを取得
2. CoreNLPの準備

In [2]:
list_sentence = Filer.readtsv("../../../大学院/データセット/OpinRankDatasetWithJudgments/cars_all_review_2009_random1000_sep.tsv")

proc = CoreNLP(configdict={'annotators': 'tokenize,ssplit,pos, lemma'}, corenlp_jars=["/usr/local/lib/stanford-corenlp-full-2015-04-20/*"])

1. 形態素解析を行って、NN, VB, JJに限定して単語を取得

In [54]:
pattern1 = r'^NN'
pattern2 = r'^VB'
pattern3 = r'^JJ'

list_sentence_sep=[]
list_sentence_rev = []
for row in list_sentence[0:600]:
    list_user_tmp = []
    list_sentence_tmp = []
    for sentence in row:
        list_sen_tmp = []
        try:
            dict_tmp = proc.parse_doc(sentence)
            for lemma, pos in zip(dict_tmp[u'sentences'][0][u'lemmas'], dict_tmp[u'sentences'][0][u'pos']):
                if re.match(pattern1, pos) or re.match(pattern2, pos) or re.match(pattern3, pos):
                    list_sen_tmp.append(lemma)
            list_user_tmp.append(list_sen_tmp)
            list_sentence_tmp.append(sentence)
        except UnicodeError:
            print "error"
    list_sentence_sep.append(list_user_tmp)
    list_sentence_rev.append(list_sentence_tmp)

error
error


1. 出現頻度300以上、2以下の単語は除去語として、登録
2. 除去語を形態素解析した結果から削除

In [55]:
list_tmp = [sen for row1 in list_sentence_sep for row2 in row1 for sen in row2]
list_count = sorted(collections.Counter(list_tmp).items(), key=lambda x: x[1], reverse=True)
list_remove = [row[0] for row in list_count if row[1] >= 300 or row[1] <= 2]

list_sentence_sep_rev = []
for row1 in list_sentence_sep:
    list_sentence_tmp = []
    for row2 in row1:
        list_tmp = []
        for word in row2:
            if word not in list_remove:
                list_tmp.append(word.encode("utf-8"))
        list_sentence_tmp.append(list_tmp)
    list_sentence_sep_rev.append(list_sentence_tmp)

1. userID, sentenceID, sentence, sep_wordsという形式のリストを作成
2. tsv形式でファイルを保存

In [ ]:
list_all = []
counter_user = 0
counter_sen = 0
for row_sen, row_sep in zip(list_sentence_rev, list_sentence_sep_rev):
    counter_sen = 0
    for sen, sep in zip(row_sen, row_sep):
        list_tmp = [counter_user, counter_sen, sen]
        list_tmp.extend(sep)
        counter_sen += 1
        list_all.append(list_tmp)
    counter_user += 1
    
Filer.writetsv(list_all, "./files/edmunds_car/list_sentence_sep.tsv")

1. UserID, sentenceID, sentence, sepを記録したtsvファイルを読み込む
2. sepの部分のみ抜き出す
3. itertoolsを使ってエッジリストを作成
4. csvファイルとしてエッジリストを保存

In [69]:
list_all = Filer.readtsv("./files/edmunds_car/list_sentence_sep.tsv")

list_all = [row[3:] for row in list_all]

list_edgelist = []
for row in list_all:
    list_tmp = list(itertools.combinations(row,2))
    list_edgelist.extend(list_tmp)
    
Filer.writecsv(list_edgelist, "./files/edmunds_car/list_edgelist_edmunds_co.csv")

1. UserID, sentenceID, sentence, sepを記録したtsvファイルを読み込む
2. sepの部分のみ抜き出す
3. UM用の学習ファイルとして、txtファイルとして保存

In [3]:
list_all = Filer.readtsv("./files/edmunds/list_sentence_sep.tsv")

list_all = [" ".join(row[3:]) for row in list_all]

Filer.writetxt(list_all, "./files/edmunds/edmunds_corpus_for_UM.txt")

1. UserID, sentenceID, sentence, sepを記録したtsvファイルを読み込む
2. UserID, sentenceID, sentenceの部分のみ抜き出す
3. sentencesIDを付け直す
4. tsvファイルに保存する

In [7]:
list_all = Filer.readtsv("./files/edmunds/list_sentence_sep.tsv")

list_all = [row[:3] for row in list_all]

list_all_rev = []
for i, row in enumerate(list_all):
    list_all_rev.append([row[0], i, row[2]])
    
Filer.writetsv(list_all_rev, "./files/edmunds/edmunds_annotation.tsv")

1. UserID, sentenceID, sentence, sepを記録したtsvファイルを読み込む
2. sentencesIDを付け直す
3. tsvファイルに保存する

In [3]:
list_all = Filer.readtsv("./files/edmunds/list_sentence_sep.tsv")

for i, row in enumerate(list_all):
    list_all[i][1] = i
    
Filer.writetsv(list_all, "./files/edmunds/list_sentence_sep_rev.tsv")